# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
# Name: Yaki Dror
# Details Student 2:


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

### PreprocessText function:
prepares input text for analysis by:
1. Converting to lowercase.
2. Removing non-Hebrew, non-English characters.
3. Eliminating extra spaces and line breaks.

In [11]:
def PreprocessText(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters, numbers, and symbols
    text = re.sub(r'[^\u0590-\u05FFa-zA-Z\s]', '', text)
    
    # Remove extra spaces and line breaks
    text = ' '.join(text.split())
 
    return text

### TrainGenderClassifier Function:
Train a gender classification model based on the provided training data by performing the following steps:

1. Load Training Data - Reads the training data from a CSV file using the provided filename.

2. Preprocess Text - Applies a preprocessing function (PreprocessText) to clean and prepare the text data within the 'story' column of the training data.

3. Feature Extraction - Utilizes TF-IDF (Term Frequency-Inverse Document Frequency) vectorization to convert the preprocessed text data into numerical feature vectors.

4. Label Encoding - Encodes the gender labels ('m' and 'f') into numeric values using label encoding.

5. Hyperparameter Tuning - Performs hyperparameter tuning for the Linear Support Vector Classifier (LinearSVC) using a grid search with cross-validation. It searches for the optimal 'C' parameter value among [0.1, 1, 10] to improve model performance.

6. Model Training - Trains the LinearSVC model with the best hyperparameters identified from the grid search.

7. Output - Returns the trained classifier (best_svc), TF-IDF vectorizer (tfidf_vectorizer), and label encoder (label_encoder) for later use in gender prediction.

##### Overall, the function prepares and trains a gender classification model using the provided training data, employing preprocessing, feature extraction, label encoding, and hyperparameter tuning techniques to achieve improved performance.

In [12]:
def TrainGenderClassifier(train_filename):
    # Load the training data
    df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
    
    # Preprocess the text
    df_train['preprocessed_text'] = df_train['story'].apply(PreprocessText)
    
    # Feature Extraction using TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['preprocessed_text'])
    
    # Label Encoding
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(df_train['gender'])
    
    # Hyperparameter tuning for LinearSVC
    param_grid = {'C': [0.1, 1, 10]}
    svc = LinearSVC(max_iter=10000)
    grid_search = GridSearchCV(svc, param_grid, cv=5)
    grid_search.fit(X_train_tfidf, y_train_encoded)
    
    # Get the best classifier from the grid search
    best_svc = grid_search.best_estimator_
    
    return best_svc, tfidf_vectorizer, label_encoder

### EvaluateModel Function:
The EvaluateModel function assesses a trained gender classification model's performance using various metrics, including precision, recall, and F1 scores for both male and female gender predictions. It calculates an average F1 score for overall evaluation.

In [13]:
def EvaluateModel(train_filename):
    # Train the model
    trained_classifier, trained_tfidf_vectorizer, trained_label_encoder = TrainGenderClassifier(train_filename)
    
    # Load the training data
    df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
    
    # Preprocess the text in the training data
    df_train['preprocessed_text'] = df_train['story'].apply(PreprocessText)
    
    # Feature Extraction using TF-IDF
    X_train_tfidf = trained_tfidf_vectorizer.transform(df_train['preprocessed_text'])
    
    # Predict gender using the trained model
    y_train_pred_encoded = trained_classifier.predict(X_train_tfidf)
    y_train_pred = trained_label_encoder.inverse_transform(y_train_pred_encoded)
    
    # Actual genders from the train data
    y_train_actual = df_train['gender']
    
    # Calculate precision, recall, and F1 score
    precision = precision_score(y_train_actual, y_train_pred, average='weighted')
    recall = recall_score(y_train_actual, y_train_pred, average='weighted')
    f1 = 2 * (precision * recall) / (precision + recall)
    
    precision_male = precision_score(y_train_actual, y_train_pred, average='binary', pos_label='m')
    recall_male = recall_score(y_train_actual, y_train_pred, average='binary', pos_label='m')

    # Calculate precision and recall for female as positive class and male as negative class
    precision_female = precision_score(y_train_actual, y_train_pred, average='binary', pos_label='f')
    recall_female = recall_score(y_train_actual, y_train_pred, average='binary', pos_label='f')

    # Calculate F1 score for male as positive class and female as negative class
    f1_male = 2 * (precision_male * recall_male) / (precision_male + recall_male)

    # Calculate F1 score for female as positive class and male as negative class
    f1_female = 2 * (precision_female * recall_female) / (precision_female + recall_female)
    
    # Calculate average F1 score
    average_f1 = (f1_male + f1_female) / 2
    
    #print("Actual Genders:", y_train_actual.tolist())
    #print("Predicted Genders:", y_train_pred.tolist())
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("F1 Score (Male):", f1_male)
    print("F1 Score (Female):", f1_female)
    print("Average F1 Score:", average_f1)
    
# Evaluate the model on the training data
EvaluateModel('annotated_corpus_for_train.csv')

Precision: 0.998674284344105
Recall: 0.99867197875166
F1 Score: 0.9986731315465518
F1 Score (Male): 0.999131190269331
F1 Score (Female): 0.9971830985915493
Average F1 Score: 0.9981571444304401


### TestGenderClassifier Function:
Employs a trained gender classification model to predict genders based on input test data.
It preprocess the text, convert it into feature vectors, make predictions, and convert encoded predictions back to original labels.
The function returns the predicted genders for the test data.

In [14]:
def TestGenderClassifier(classifier, tfidf_vectorizer, label_encoder, test_filename):
    df_test = pd.read_csv(test_filename, index_col=None, encoding='utf-8')
    
    df_test['preprocessed_text'] = df_test['story'].apply(PreprocessText)
    
    X_test_tfidf = tfidf_vectorizer.transform(df_test['preprocessed_text'])
    
    y_pred_encoded = classifier.predict(X_test_tfidf)
    
    y_pred = label_encoder.inverse_transform(y_pred_encoded)
    
    return y_pred

### PredictGender Function:
Uses a trained classifier to predict genders for test data. It loads the classifier and related tools, makes predictions, creates a DataFrame with predictions, and displays the first and last 5 predictions.

In [15]:
def PredictGender():
    # Load the trained classifier, vectorizer, and label encoder
    trained_classifier, trained_tfidf_vectorizer, trained_label_encoder = TrainGenderClassifier('annotated_corpus_for_train.csv')

    # Use the TestGenderClassifier function to predict gender
    predictions = TestGenderClassifier(trained_classifier, trained_tfidf_vectorizer, trained_label_encoder, 'corpus_for_test.csv')

    # Create a DataFrame to store predictions
    df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_gender': predictions})
    
    # Print the first 5 and last 5 predictions
    print("First 5 predictions:")
    print(df_predicted.head())
    print("\nLast 5 predictions:")
    print(df_predicted.tail())
    
    return df_predicted

df_predicted = PredictGender()

First 5 predictions:
   test_example_id predicted_gender
0                0                m
1                1                m
2                2                m
3                3                m
4                4                m

Last 5 predictions:
     test_example_id predicted_gender
318              318                m
319              319                m
320              320                m
321              321                m
322              322                m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [16]:
df_predicted.to_csv('classification_results.csv',index=False)